***GENERATED CODE FOR personalizedrecommendations PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ClassName', 'transformation_label': 'String Indexer'}], 'feature': 'ClassName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'BI BootCamp  Feb 22 2014 Richardson', 'max': 'SQLBI Bootcamp  May 22 2021 ONLINE', 'missing': '0', 'distinct': '77'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ClassName'}, {'feature_label': 'ClassName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ClassName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ClassType', 'transformation_label': 'String Indexer'}], 'feature': 'ClassType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Live In  Person', 'max': 'Self Paced Online', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ClassType'}, {'feature_label': 'ClassType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ClassType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Education', 'transformation_label': 'String Indexer'}], 'feature': 'Education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '   year of college', 'max': 'Some graduate school', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Education'}, {'feature_label': 'Education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'HouseholdIncome', 'transformation_label': 'String Indexer'}], 'feature': 'HouseholdIncome', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '  ', 'max': '  ', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'HouseholdIncome'}, {'feature_label': 'HouseholdIncome', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('HouseholdIncome')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'No  EnrollReason', 'transformation_label': 'String Indexer'}], 'feature': 'No  EnrollReason', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Advertising', 'max': 'the way tuition is paid. its a game changer to older school models.', 'missing': '0', 'distinct': '31'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'No  EnrollReason'}, {'feature_label': 'No  EnrollReason', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('No  EnrollReason')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Age', 'transformation_label': 'String Indexer'}], 'feature': 'Age', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '36.95', 'stddev': '8.71', 'min': '21', 'max': 'None', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Age'}, {'feature_label': 'Age', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Age')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'None', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CountryOfOrgin', 'transformation_label': 'String Indexer'}], 'feature': 'CountryOfOrgin', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Burundi', 'max': 'Zimbabwe', 'missing': '0', 'distinct': '25'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CountryOfOrgin'}, {'feature_label': 'CountryOfOrgin', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CountryOfOrgin')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'JobRole', 'transformation_label': 'String Indexer'}], 'feature': 'JobRole', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Accountant', 'max': 'unemployeed', 'missing': '0', 'distinct': '93'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'JobRole'}, {'feature_label': 'JobRole', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('JobRole')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WhyChooseForCareerGoals', 'transformation_label': 'String Indexer'}], 'feature': 'WhyChooseForCareerGoals', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '   To fill the gap in knowledge what the employer needs and what I have    Communication challenges specially in interviews', 'max': 'yes', 'missing': '0', 'distinct': '416'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'WhyChooseForCareerGoals'}, {'feature_label': 'WhyChooseForCareerGoals', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('WhyChooseForCareerGoals')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Industry', 'transformation_label': 'String Indexer'}], 'feature': 'Industry', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Advertising and MARCHketing', 'max': 'Utilities Energy and Extraction', 'missing': '0', 'distinct': '22'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Industry'}, {'feature_label': 'Industry', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Industry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InformationConsumed', 'transformation_label': 'String Indexer'}], 'feature': 'InformationConsumed', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Daily Emails\xa0', 'max': 'Weekly Emails\xa0', 'missing': '0', 'distinct': '14'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InformationConsumed'}, {'feature_label': 'InformationConsumed', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InformationConsumed')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import optuna
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()

    def objective(trial):
        n_clusters = int(trial.suggest_loguniform("n_clusters", 2, 10))
        classifier_obj = KMeans(n_clusters=n_clusters).fit(pd_df)
        labels = classifier_obj.predict(pd_df)
        score = metrics.silhouette_score(pd_df, labels, metric='euclidean')
        return score
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    best = study.best_params
    best_param = int(best["n_clusters"])
    model = KMeans(n_clusters=best_param).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        autokmeans_obj = kmeans(df=spark_DF, TrainingPercent=trainingPercent)
        [autokmeans_obj]


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run personalizedrecommendationsHooks.ipynb
try:
	#sourcePreExecutionHook()

	demographicsdatacolaberrypersonalized = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/demographicsdatacolaberrypersonalized.csv', 'filename': 'demographicsdatacolaberrypersonalized.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(demographicsdatacolaberrypersonalized)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run personalizedrecommendationsHooks.ipynb
try:
	#transformationPreExecutionHook()

	personalizedrecommendationsautofe = TransformationMain.run(demographicsdatacolaberrypersonalized,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ClassName", "transformation_label": "String Indexer"}], "feature": "ClassName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "BI BootCamp  Feb 22 2014 Richardson", "max": "SQLBI Bootcamp  May 22 2021 ONLINE", "missing": "0", "distinct": "77"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ClassName"}, {"transformationsData": [{"feature_label": "ClassType", "transformation_label": "String Indexer"}], "feature": "ClassType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Live In  Person", "max": "Self Paced Online", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ClassType"}, {"transformationsData": [{"feature_label": "Education", "transformation_label": "String Indexer"}], "feature": "Education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "   year of college", "max": "Some graduate school", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Education"}, {"transformationsData": [{"feature_label": "HouseholdIncome", "transformation_label": "String Indexer"}], "feature": "HouseholdIncome", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "  ", "max": "  ", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "HouseholdIncome"}, {"transformationsData": [{"feature_label": "No  EnrollReason", "transformation_label": "String Indexer"}], "feature": "No  EnrollReason", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Advertising", "max": "the way tuition is paid. its a game changer to older school models.", "missing": "0", "distinct": "31"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "No  EnrollReason"}, {"transformationsData": [{"feature_label": "Age", "transformation_label": "String Indexer"}], "feature": "Age", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "36.95", "stddev": "8.71", "min": "21", "max": "None", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "None", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "CountryOfOrgin", "transformation_label": "String Indexer"}], "feature": "CountryOfOrgin", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Burundi", "max": "Zimbabwe", "missing": "0", "distinct": "25"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CountryOfOrgin"}, {"transformationsData": [{"feature_label": "JobRole", "transformation_label": "String Indexer"}], "feature": "JobRole", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Accountant", "max": "unemployeed", "missing": "0", "distinct": "93"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "JobRole"}, {"transformationsData": [{"feature_label": "WhyChooseForCareerGoals", "transformation_label": "String Indexer"}], "feature": "WhyChooseForCareerGoals", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "   To fill the gap in knowledge what the employer needs and what I have    Communication challenges specially in interviews", "max": "yes", "missing": "0", "distinct": "416"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "WhyChooseForCareerGoals"}, {"transformationsData": [{"feature_label": "Industry", "transformation_label": "String Indexer"}], "feature": "Industry", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Advertising and MARCHketing", "max": "Utilities Energy and Extraction", "missing": "0", "distinct": "22"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Industry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YrsOnJob", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3.82", "stddev": "2.98", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "YrsOnJob"}, {"transformationsData": [{"feature_label": "InformationConsumed", "transformation_label": "String Indexer"}], "feature": "InformationConsumed", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Daily Emails\u00a0", "max": "Weekly Emails\u00a0", "missing": "0", "distinct": "14"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InformationConsumed"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NetPromotorScore", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "8.48", "stddev": "1.85", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "NetPromotorScore"}]}))

	#transformationPostExecutionHook(personalizedrecommendationsautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run personalizedrecommendationsHooks.ipynb
try:
	#mlPreExecutionHook()

	personalizedrecommendationsclustering = Clustering.run(personalizedrecommendationsautofe,spark,json.dumps( {"autoClustering": 1, "originalfile": "/FileStore/platform/uploadedSourceFiles/demographicsdatacolaberrypersonalized.csv", "model": "Kmeans", "TrainingPercent": 100, "totalClusters": 2}))

	#mlPostExecutionHook(personalizedrecommendationsclustering)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
